## Extraction the Harry Potter related entities

- Characters: Harry Potter, Hermione Granger, Albus Dumbledore, etc.

- Locations: Hogwarts, Diagon Alley, Hogsmeade, etc.

- Organizations: Ministry of Magic, Order of the Phoenix, Death Eaters, etc.

- Magical Objects: Elder Wand, Invisibility Cloak, Philosopher's Stone, etc.

- Spells and Potions: Expelliarmus, Polyjuice Potion, etc.

- Events: Triwizard Tournament, Battle of Hogwarts, etc.

- Creatures: Hippogriff, Basilisk, Dementor, etc.

In [ ]:
import requests
import re
import pandas as pd
import json
from bs4 import BeautifulSoup

In [53]:
# Characters
characters = requests.get("https://hp-api.onrender.com/api/characters").json()
character_names = [character["name"] for character in characters]
with open('Character_Names.json', 'w') as file:
    json.dump(character_names, file)


In [54]:
# Magical objects

magical_objects_data = []
url = "https://harrypotter.fandom.com/wiki/Category:Magical_objects"

# Send GET request
response = requests.get(url, headers=headers)
response.raise_for_status()  # Check for HTTP errors


soup = BeautifulSoup(response.text, 'html.parser')
contents = soup.find_all('li', {'class': 'category-page__member'})
for content in contents:
    magical_object = content.text.strip()
    magical_objects_data.append(magical_object)


url = "https://harrypotter.fandom.com/wiki/Category:Magical_objects?from=Omnioculars"


# Send GET request
response = requests.get(url, headers=headers)
response.raise_for_status()  # Check for HTTP errors


soup = BeautifulSoup(response.text, 'html.parser')
contents = soup.find_all('li', {'class': 'category-page__member'})
for content in contents:
    magical_object = content.text.strip()
    magical_objects_data.append(magical_object)

print(len(magical_objects_data))
with open('Magical_Objects.json', 'w') as file:
    json.dump(magical_objects_data, file)

311


In [55]:
# Spells
spells = requests.get("https://hp-api.onrender.com/api/spells").json()
spell_names = [spell["name"] for spell in spells]
with open('Spell_Names.json', 'w') as file:
    json.dump(spell_names, file)

In [56]:
# Potions
potions_data = []

url = "https://harrypotter.fandom.com/wiki/List_of_potions"
# Send GET request
response = requests.get(url, headers=headers)
response.raise_for_status()  # Check for HTTP errors
soup = BeautifulSoup(response.text, 'html.parser')


contents = soup.find_all('div', {'class':'wds-tab__content'})
for content in contents:
    for ul in content.find_all('ul',recursive=False):
        potions = ul.find_all('li',recursive=False)
        for potion in potions:
            potions_data.append(potion.find('a').text.strip())
       
print(len(potions_data))
print(potions_data)
with open('Potions.json', 'w') as file:
    json.dump(potions_data, file)


99
['Ageing Potion', 'Alihotsy Draught', 'Amortentia', "Angel's Trumpet Draught", 'Anti-Paralysis Potion', 'Antidote to Common Poisons', 'Antidote to Uncommon Poisons', 'Antidote to Veritaserum', 'Armadillo Bile Mixture', 'Babbling Beverage', 'Baneberry Potion', "Baruffio's Brain Elixir", 'Beautification Potion', 'Befuddlement Draught', 'Beguiling Bubbles', 'Black Fire Potion', 'Blemish Blitzer', 'Blood-Replenishing Potion', 'Bloodroot Poison', 'Bruise removal paste', 'Bulgeye Potion', 'Bundimun Pomade', 'Bundimun Secretion', 'Burn-Healing Paste', 'Burning Bitterroot Balm', 'Draught of Living Death', 'Draught of Peace', 'Dreamless Sleep Potion', 'Drink of Despair', 'Drowsiness Draught', 'Edurus Potion', 'Elixir 7', 'Elixir of Life', 'Elixir to Induce Euphoria', 'Emerald Potion', 'Erumpent Potion', 'Everklena', 'Essence of Insanity', 'Everlasting Elixirs', 'Exploding Potion', 'Exstimulo Potion', 'Fake Protective Potions', 'Fatiguing Infusion', 'Felix Felicis', 'Fergus Fungal Budge', 'Fi

In [57]:
# creatures
url = "https://harrypotter.fandom.com/wiki/List_of_creatures"


# Send GET request
response = requests.get(url, headers=headers)
response.raise_for_status()  # Check for HTTP errors

soup = BeautifulSoup(response.text, 'html.parser')
content = soup.find('div', {'class': 'mw-parser-output'})

creatures_data = []

# Find all section headings (h2 with span.mw-headline)
for section in content.find_all('h2'):
    # Extract section title
    headline = section.find('span', class_='mw-headline')
    if not headline:
        continue
    
    section_title = headline.text.strip()
    # print(section_title)
    
    # Skip non-creature sections
    if section_title == 'Notes and references':
        continue
    
    # Find the corresponding creatures list
    creatures_ul = section.find_next('ul', )
    # print(creatures_ul)
    if not creatures_ul:
        continue
    
    # Extract creature names
    for li in creatures_ul.find_all('li'):

        li_ul = li.find_all('ul')
        if li_ul:
            for li_li in li_ul:
                if isinstance(li_li, list):
                    for sub_li in li_li.find_all('li'):
                        creature = sub_li.text.strip()
                        # Remove reference numbers like [1], [5], etc.
                        creature = re.sub(r'\[\d+\]', '', creature)
                        creature = re.sub(r'\s*\(.*?\)', '', creature)
                        creatures_data.append({
                            'Section': section_title,
                            'Creature': creature
                        })
                else:
                    creature = li_li.text.strip()
                    # Remove reference numbers like [1], [5], etc.
                    creature = re.sub(r'\[\d+\]', '', creature)
                    creature = re.sub(r'\s*\(.*?\)', '', creature)
                    creatures_data.append({
                        'Section': section_title,
                        'Creature': creature
                    })
            next
        creature = li.text.strip()
        # Remove reference numbers like [1], [5], etc.
        creature = re.sub(r'\[\d+\]', '', creature)
        creature = re.sub(r'\s*\(.*?\)', '', creature)
        creatures_data.append({
            'Section': section_title,
            'Creature': creature
        })

# drop duplicates by "Creature" column
creatures_df = pd.DataFrame(creatures_data)
creatures_df.drop_duplicates(subset='Creature', inplace=True)
creatures_df.reset_index(drop=True, inplace=True)
creatures_df_cleaned = creatures_df[~creatures_df['Creature'].str.contains('\n', na=False)]
creatures_df_cleaned.reset_index(drop=True, inplace=True)
print(creatures_df_cleaned)

creatures_df_cleaned.to_json('Creatures.json', orient='records', lines=True)

                   Section        Creature
0                   Beasts     Acromantula
1                   Beasts       Ashwinder
2                   Beasts         Augurey
3                   Beasts        Basilisk
4                   Beasts        Billywig
..                     ...             ...
284  Magic related animals   Tropical bird
285  Magic related animals  Tarantula hawk
286  Magic related animals       Wild Boar
287  Magic related animals        The Grim
288  Magic related animals  The White Wolf

[289 rows x 2 columns]
